In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

c:\Users\nikhi\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
api_key = "GEMINI API KEY"



In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=api_key,
    temperature=0.7
)


In [7]:
poem = llm.invoke("Write a poem on dosa")

print(poem.content)

A ladle dips in fermented white,
Of rice and lentils, aged just right.
The tawa glows with a steady heat,
To craft a savory, golden treat.

A splash of water, a sudden hiss,
The kitchen smells of morning bliss.
The batter falls, a rhythmic swirl,
As edges start to crisp and curl.

From center out, a perfect ring,
A paper-thin and lacy thing.
A brush of ghee, a glint of oil,
The reward for the evening’s toil.

It turns a shade of toasted brown,
The jewel of the breakfast crown.
Perhaps a scoop of potato mash,
With mustard seeds and a chili flash.

Beside it sits the coconut white,
And tomato chutney, red and bright.
A bowl of sambar, steaming hot,
To hit the perfect, tangy spot.

With a gentle snap and a tender bite,
It brings a sense of pure delight.
Plain or masala, long or small,
The mighty dosa rules them all.


In [13]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "atliq_tshirts"

db =SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info = 3)

print(db.get_table_info())


CREATE TABLE discounts (
	discount_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER(11) NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER(11) NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER(11), 
	stock_quantity INTEGER(11) NOT NULL, 
	PRIMARY KEY (t_shirt_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	M	46	16
2	Nike	Red	XL	19	37
3	Adidas	White	S	11	12
*/


In [17]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

qnsl = db_chain.run("How many t-shirt do we have left for nike in extra small size and white color?")

C:\Users\nikhi\AppData\Local\Temp\ipykernel_15540\2095684386.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qnsl = db_chain.run("How many t-shirt do we have left for nike in extra small size and white color?")




> Entering new SQLDatabaseChain chain...
How many t-shirt do we have left for nike in extra small size and white color?
SQLQuery:SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
SQLResult: [(Decimal('67'),)]
Answer:67
> Finished chain.


In [18]:
qns2 = db_chain.run("How much the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much the price of the inventory for all small size t-shirts?
SQLQuery:SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('30805'),)]
Answer:The total price of the inventory for all small size t-shirts is 30,805.
> Finished chain.


In [19]:
qns3 = db_chain.run("If we have to sell all the Levi's T-shirts today with discount applied. How much store will generate?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's T-shirts today with discount applied. How much store will generate?
SQLQuery:SQLQuery: SELECT SUM(`price` * (1 - IFNULL(`pct_discount`, 0) / 100) * `stock_quantity`) FROM `t_shirts` LEFT JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id` WHERE `brand` = 'Levi'
SQLResult: [(Decimal('18913.400000'),)]
Answer:Answer: The store will generate 18913.4 if all Levi's T-shirts are sold with the applied discounts.
> Finished chain.


In [22]:
qsn1 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color ='White' AND size='XS'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color ='White' AND size='XS'
SQLQuery:Question: SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color ='White' AND size='XS'
SQLQuery: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `color` = 'White' AND `size` = 'XS'
SQLResult: [(Decimal('67'),)]
Answer:67
> Finished chain.


Few shot learning


In [27]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "67"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "30805"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "18913.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "18987"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "how much sales amount will be generated if we sell all large size t shirts today in nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "5773.00"
    }
]

In [45]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

#e = embeddings.embed_query("How many white color Levi's shirt I have?")

In [24]:
len(e)

384

In [25]:
e[:5]

[0.028084732592105865,
 0.12849895656108856,
 0.0052041662856936455,
 0.013358933851122856,
 0.043256718665361404]

In [46]:
to_vectorize = ["" .join(example.values()) for example in few_shots]

In [47]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'Result of the SQL query67",
 "How much is the total price of the inventory for all S-size t-shirts?SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'Result of the SQL query30805",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n Result of the SQL query18913.4",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'Result of t

In [48]:
#creating vector database

from langchain.vectorstores import Chroma 

vectorstore = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [49]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_selector.select_examples({"Question": "How many adidas t shirts i have left in my store?"})

[{'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '67',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?'},
 {'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '67',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?'}]

In [37]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt


In [38]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [39]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [40]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables= ["Question", "SQLQuuery", "SQLResult", "Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",

)

In [51]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt =FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = _mysql_prompt,
    suffix = PROMPT_SUFFIX,
    input_variables = ["input","table_info","top_k"],
)

In [52]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [53]:
new_chain("How many white Levi's shirt i have?")



> Entering new SQLDatabaseChain chain...
How many white Levi's shirt i have?
SQLQuery:SQLQuery: SELECT sum(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('290'),)]
Answer:290
> Finished chain.


{'query': "How many white Levi's shirt i have?", 'result': '290'}

In [54]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S'
SQLResult: [(Decimal('30805'),)]
Answer:30805
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '30805'}